In [1]:
# Import libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import glob
import os
import datetime
from datetime import time
from datetime import timedelta
from datetime import datetime

# Import plotly express for EF plot
# import plotly.express as px
# import plotly.graph_objects as go
# px.defaults.width, px.defaults.height = 1000,600
# Set precision
pd.set_option('display.precision', 4)

In [2]:
df4 = pd.read_csv('options_final_total_1.csv', index_col=0)

In [3]:
# Convert 'quote_datetime' to datetime
df4['quote_datetime'] = pd.to_datetime(df4['quote_datetime'])

In [4]:
df4['date'] = df4['quote_datetime'].dt.date

In [5]:
# df6 = df4.groupby('date').tail(1).reset_index(drop=True)

In [6]:
# df6.to_excel('straddle932.xlsx')

In [7]:
# Specify the columns you want to import
column_headers = ['futures_Date', 'futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']

# Import only the OHLC columns
df_spot = pd.read_csv('ES_full_1min.txt', names=column_headers)

# Ensure the 'Date' column is in datetime format
df_spot['futures_Date'] = pd.to_datetime(df_spot['futures_Date'])

# Sort the DataFrame by the 'Date' column
df_spot = df_spot.sort_values(by='futures_Date').reset_index(drop=True)

# Replace commas and convert columns to float
df_spot[['futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']] = df_spot[['futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']].replace({',': ''}, regex=True).astype(float)

In [8]:
# Add 1 minute to every value in the 'Date' column
df_spot['futures_Date_plus_1min'] = df_spot['futures_Date'] + pd.DateOffset(minutes=1)

In [9]:
del df_spot['futures_Date']
del df_spot['futures_Volume']

In [10]:
# Filter rows where the time falls between 9:35 AM and 4:10 PM
start_time = pd.to_datetime('09:31:00').time()
end_time = pd.to_datetime('16:00:00').time()

# Apply the filter based on the time component of 'quote_datetime'
df_spot = df_spot[(df_spot['futures_Date_plus_1min'].dt.time >= start_time) & (df_spot['futures_Date_plus_1min'].dt.time <= end_time)]

In [11]:
df_spot = df_spot.reset_index(drop=True)

In [12]:
df_spot['intraday_sma_20'] = (
    df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date)['futures_Close']
    .transform(lambda x: x.rolling(window=20, min_periods=1).mean())
)

In [13]:
# Group by date and calculate the number of unique times per date
df_spot = df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date).filter(
    lambda x: len(x['futures_Date_plus_1min'].dt.time.unique()) == 390
)

In [14]:
df4.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932'],
      dtype='object')

In [15]:
df_spot['futures_Low_min'] = df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date)['futures_Low'].cummin()

In [16]:
# df_spot[df_spot['futures_Date_plus_1min'].dt.date == pd.to_datetime('2018-01-16').date()].to_csv('ch.csv')

In [17]:
df = pd.merge(df4, df_spot, left_on =  'quote_datetime', right_on = 'futures_Date_plus_1min', how = 'left')

In [18]:
df.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-01-02 09:32:00,2685.0,3.30,4.3,3.00,3.9,2685.60,2685.60,2684.30,2684.30,...,4.1,7.25,2684.3,2685.50,2685.50,2684.00,2684.00,2018-01-02 09:32:00,2684.6250,2684.00
1,2018-01-02 09:33:00,2685.0,3.90,3.9,3.40,3.4,2684.32,2685.74,2684.23,2685.30,...,4.1,7.25,2684.3,2684.00,2685.75,2684.00,2685.00,2018-01-02 09:33:00,2684.7500,2684.00
2,2018-01-02 09:34:00,2685.0,3.10,4.5,2.70,4.2,2685.18,2685.18,2683.73,2683.73,...,4.1,7.25,2684.3,2685.00,2685.00,2683.25,2683.50,2018-01-02 09:34:00,2684.4375,2683.25
3,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,4.1,7.25,2684.3,2683.50,2684.00,2682.50,2682.75,2018-01-02 09:35:00,2684.1000,2682.50
4,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,4.1,7.25,2684.3,2682.75,2683.00,2682.25,2682.75,2018-01-02 09:36:00,2683.8750,2682.25


In [19]:
# Calculate the number of unique days
num_unique_days = df['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1320


In [20]:
df['date'] = df['quote_datetime'].dt.date

In [21]:
# Identify dates where all `straddle_exit` values are >= 0 for every ticker
valid_dates = df.groupby('date')['straddle_exit'].apply(lambda x: (x >= 0).all())

In [22]:
# Filter `df_atm` to keep only rows from the valid dates
df = df[df['date'].isin(valid_dates.index)]

In [23]:
df['time'] = df['quote_datetime'].dt.time

In [24]:
# #Read the CSV file
# dates = pd.read_excel('E:\marker2nitesh.xlsx')

# #Extract the 'Date' column (replace 'Date' with the actual column name in your CSV)
# dates_to_plot = dates['date'].astype(str).tolist()

In [25]:
# dates_to_plot = pd.to_datetime(dates_to_plot).date
# # Filter the DataFrame for the specified dates
# df = df[df['date'].isin(dates_to_plot)]

In [26]:
# Calculate the number of unique days
num_unique_days = df['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1320


In [27]:
df.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'futures_Open', 'futures_High', 'futures_Low', 'futures_Close',
       'futures_Date_plus_1min', 'intraday_sma_20', 'futures_Low_min'],
      dtype='object')

In [28]:
df = df.reset_index(drop=True)

In [29]:
# df = df[df['date'] == pd.to_datetime('2024-04-16').date()]

In [30]:
# df.head(14)

In [31]:
def filter_based_on_highs(df):
    # Initialize an empty DataFrame to store results
    filtered_df = pd.DataFrame()

    # Group by each date
    for date, group in df.groupby('date'):
        # Sort by time to ensure correct order
        group = group.sort_values(by='time').reset_index(drop=True)

        # Identify the highest high of the first three candles
        first_three_high = group.loc[
            group['time'].isin([time(9, 32), time(9, 33), time(9, 34)]), 'futures_High'
        ].max()
        current_high = first_three_high
        filter_start_index = None
        high_updated_once = False  # Flag to track if `current_high` has been updated once

        # Iterate through the subsequent candles
        for idx, row in group.iterrows():
            if row['time'] in [time(9, 32), time(9, 33), time(9, 34)]:
                continue  # Skip the first three candles already processed
            
            if row['futures_Close'] > current_high:
                # Mark the index to start filtering from here
                filter_start_index = idx
                break
            elif not high_updated_once and row['futures_High'] > current_high:
                # Update the highest high only once
                current_high = row['futures_High']
                high_updated_once = True
        
        # Filter the DataFrame from the identified starting point
        if filter_start_index is not None:
            filtered_df = pd.concat([filtered_df, group.iloc[filter_start_index:]])
    
    # Reset index for the final DataFrame
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [32]:
# Apply the function to your DataFrame
df5 = filter_based_on_highs(df)

In [33]:
# Calculate the number of unique days
num_unique_days = df5['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1104


In [34]:
df5.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-01-02 09:41:00,2685.0,4.5,2.75,4.2,2.50,2685.73,2686.90,2685.73,2686.57,...,4.1,7.25,2684.3,2685.75,2686.75,2685.75,2686.25,2018-01-02 09:41:00,2684.2500,2681.75
1,2018-01-02 09:42:00,2685.0,4.7,2.60,3.9,2.40,2686.56,2686.99,2686.41,2686.80,...,4.1,7.25,2684.3,2686.25,2687.00,2686.25,2686.75,2018-01-02 09:42:00,2684.4583,2681.75
2,2018-01-02 09:43:00,2685.0,4.7,2.55,4.2,2.35,2686.84,2687.13,2686.55,2686.95,...,4.1,7.25,2684.3,2686.75,2686.75,2686.25,2686.75,2018-01-02 09:43:00,2684.6346,2681.75
3,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,4.1,7.25,2684.3,2686.50,2686.75,2685.75,2686.00,2018-01-02 09:44:00,2684.7321,2681.75
4,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,4.1,7.25,2684.3,2686.00,2686.50,2685.50,2685.75,2018-01-02 09:45:00,2684.8000,2681.75


In [35]:
def filter_low_break(df):
    # Initialize an empty DataFrame to store results
    filtered_df = pd.DataFrame()

    # Group by each date
    for date, group in df.groupby('date'):
        # Sort by time to ensure correct order
        group = group.sort_values(by='time').reset_index(drop=True)

        # Initialize a variable to store the previous low
        previous_low = None
        filter_start_index = None

        # Iterate through the DataFrame
        for idx, row in group.iterrows():
            if previous_low is not None and row['futures_Close'] < previous_low:
                # Mark the index to start filtering
                filter_start_index = idx
                break

            # Update the previous low for the next iteration
            previous_low = row['futures_Low']

        # Filter the DataFrame from the identified starting point
        if filter_start_index is not None:
            filtered_df = pd.concat([filtered_df, group.iloc[filter_start_index:]])

    # Reset index for the final DataFrame
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [36]:
df6 = filter_low_break(df5)

In [37]:
df6.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,4.1,7.25,2684.3,2686.5,2686.75,2685.75,2686.00,2018-01-02 09:44:00,2684.7321,2681.75
1,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,4.1,7.25,2684.3,2686.0,2686.50,2685.50,2685.75,2018-01-02 09:45:00,2684.8000,2681.75
2,2018-01-02 09:46:00,2685.0,3.6,3.40,3.1,3.30,2686.09,2686.35,2685.17,2685.17,...,4.1,7.25,2684.3,2686.0,2686.00,2684.50,2684.50,2018-01-02 09:46:00,2684.7812,2681.75
3,2018-01-02 09:47:00,2685.0,4.1,3.00,3.6,2.80,2684.94,2685.85,2684.94,2685.75,...,4.1,7.25,2684.3,2684.5,2685.75,2684.50,2685.50,2018-01-02 09:47:00,2684.8235,2681.75
4,2018-01-02 09:48:00,2685.0,3.8,3.00,3.5,2.85,2685.74,2685.90,2685.27,2685.62,...,4.1,7.25,2684.3,2685.5,2685.75,2684.75,2685.50,2018-01-02 09:48:00,2684.8611,2681.75


In [38]:
# Ensure 'date' is a column or derived appropriately
df6['selling_call_price'] = (
    df6.groupby('date')['call_price']
    .transform('first')
)
df6['selling_call_price'] = df6['selling_call_price'].ffill()

In [39]:
# Ensure 'date' is a column or derived appropriately
df6['selling_put_price'] = (
    df6.groupby('date')['put_price']
    .transform('first')
)
df6['selling_put_price'] = df6['selling_put_price'].ffill()

In [40]:
# Ensure 'date' is a column or derived appropriately
df6['selling_time'] = (
    df6.groupby('date')['time']
    .transform('first')
)
df6['selling_time'] = df6['selling_time'].ffill()

In [41]:
df6.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time
0,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,2686.5,2686.75,2685.75,2686.00,2018-01-02 09:44:00,2684.7321,2681.75,3.95,2.675,09:44:00
1,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,2686.0,2686.50,2685.50,2685.75,2018-01-02 09:45:00,2684.8000,2681.75,3.95,2.675,09:44:00
2,2018-01-02 09:46:00,2685.0,3.6,3.40,3.1,3.30,2686.09,2686.35,2685.17,2685.17,...,2686.0,2686.00,2684.50,2684.50,2018-01-02 09:46:00,2684.7812,2681.75,3.95,2.675,09:44:00
3,2018-01-02 09:47:00,2685.0,4.1,3.00,3.6,2.80,2684.94,2685.85,2684.94,2685.75,...,2684.5,2685.75,2684.50,2685.50,2018-01-02 09:47:00,2684.8235,2681.75,3.95,2.675,09:44:00
4,2018-01-02 09:48:00,2685.0,3.8,3.00,3.5,2.85,2685.74,2685.90,2685.27,2685.62,...,2685.5,2685.75,2684.75,2685.50,2018-01-02 09:48:00,2684.8611,2681.75,3.95,2.675,09:44:00


In [42]:
# end_time = pd.to_datetime('10:32:00').time()

# # Apply the filter based on the time component of 'quote_datetime'
# df6 = df6[df6['quote_datetime'].dt.time <= end_time]

In [43]:
def exit_strategy(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'Close' value for each date (first_close)
        #first_close = group['Close'].iloc[0]
        first_call = group['call_price'].iloc[0]
        
        # Step 2: Calculate the condition and filter the rows accordingly
        condition = group['call_price'] < 1.7 * first_call
        
        # Step 3: Find the first row where the condition fails and filter accordingly
        if condition.any():
            first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
            # Keep rows before the first failure (inclusive of the row where it fails)
            if first_fail_idx:
                filtered_group = group.loc[:first_fail_idx]
            else:
                filtered_group = group  # Keep the entire group if the condition never fails
            
            filtered_df_list.append(filtered_group)
    
    return pd.concat(filtered_df_list, ignore_index=True)

In [44]:
# Step 5: Apply the exit strategy
df6 = exit_strategy(df6)

In [45]:
# def exit_strategy1(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Step 1: Get the first 'Close' value for each date (first_close)
#         #first_close = group['Close'].iloc[0]
#         first_call = group['call_price'].iloc[0]
        
#         # Step 2: Calculate the condition and filter the rows accordingly
#         condition = group['call_price'] > 0.3 * first_call
        
#         # Step 3: Find the first row where the condition fails and filter accordingly
#         if condition.any():
#             first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
#             # Keep rows before the first failure (inclusive of the row where it fails)
#             if first_fail_idx:
#                 filtered_group = group.loc[:first_fail_idx]
#             else:
#                 filtered_group = group  # Keep the entire group if the condition never fails
            
#             filtered_df_list.append(filtered_group)
    
#     return pd.concat(filtered_df_list, ignore_index=True)

In [46]:
def exit_strategy1(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_put = group['put_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1.5 * first_put
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        #condition_triggered = False
        #group['put_stop_price'] = np.nan
        group['put_stop_time'] = np.nan
        
        for i in range(len(group)):
            if not stop_loss_triggered and group['put_price'].iloc[i] >= stop_loss_level:
                stop_loss_triggered = True
                a = i
                #stop_loss_time = group['time'].iloc[i] 
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group.loc[group.index[i], 'put_price'] = group.loc[group.index[a], 'put_price']
                #group['stoploss1'] = True
                group['put_stop_time'] = group.loc[group.index[a], 'time']
                #group['stoploss_time'] = stop_loss_time
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [47]:
# Step 5: Apply the exit strategy
df6 = exit_strategy1(df6)

In [48]:
df6[df6['put_stop_time'].notna()].head(8)

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time,put_stop_time
408,2018-01-10 10:43:00,2745.0,3.60,2.55,3.30,2.40,2746.44,2746.45,2745.77,2745.79,...,2747.50,2746.50,2746.50,2018-01-10 10:43:00,2745.2000,2736.5,3.45,2.475,10:43:00,10:50:00
409,2018-01-10 10:44:00,2745.0,3.40,2.75,3.10,2.55,2745.83,2745.84,2745.34,2745.34,...,2746.75,2746.00,2746.00,2018-01-10 10:44:00,2745.2625,2736.5,3.45,2.475,10:43:00,10:50:00
410,2018-01-10 10:45:00,2745.0,3.40,2.65,3.10,2.55,2745.33,2745.33,2745.04,2745.31,...,2746.25,2745.75,2746.00,2018-01-10 10:45:00,2745.3125,2736.5,3.45,2.475,10:43:00,10:50:00
411,2018-01-10 10:46:00,2745.0,3.30,2.75,3.00,2.55,2745.30,2745.36,2745.11,2745.14,...,2746.25,2745.75,2746.25,2018-01-10 10:46:00,2745.3750,2736.5,3.45,2.475,10:43:00,10:50:00
412,2018-01-10 10:47:00,2745.0,3.00,2.95,2.70,2.80,2745.13,2745.21,2744.86,2744.86,...,2746.25,2745.50,2745.50,2018-01-10 10:47:00,2745.4125,2736.5,3.45,2.475,10:43:00,10:50:00
413,2018-01-10 10:48:00,2745.0,2.70,3.30,2.55,3.00,2744.85,2744.85,2743.99,2744.14,...,2745.75,2744.50,2744.75,2018-01-10 10:48:00,2745.4250,2736.5,3.45,2.475,10:43:00,10:50:00
414,2018-01-10 10:49:00,2745.0,2.50,3.60,2.40,3.30,2744.10,2744.19,2743.77,2743.77,...,2745.25,2744.25,2744.25,2018-01-10 10:49:00,2745.4250,2736.5,3.45,2.475,10:43:00,10:50:00
415,2018-01-10 10:50:00,2745.0,2.35,3.90,2.20,3.60,2743.74,2743.81,2743.13,2743.34,...,2744.75,2743.75,2744.25,2018-01-10 10:50:00,2745.4375,2736.5,3.45,2.475,10:43:00,10:50:00


In [49]:
def exit_strategy2(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):

        # If all 'put_stop_time' values are NaN, skip the group
        if group['put_stop_time'].isna().all():
            filtered_df_list.append(group)  # Append the entire group
            continue
            
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_call = group['call_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 0.3 * first_call

        subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        #condition_triggered = False
        group['call_decay_price'] = np.nan
        group['call_decay_time'] = np.nan
        
        for i in range(len(subset)):
            if not stop_loss_triggered and subset['call_price'].iloc[i] < stop_loss_level:
                stop_loss_triggered = True
                a = i
                #stop_loss_time = group['time'].iloc[i] 
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group['call_decay_price'] = subset.loc[subset.index[a], 'call_price']
                #group['stoploss1'] = True
                group['call_decay_time'] = subset.loc[subset.index[a], 'time']
                #group['stoploss_time'] = stop_loss_time
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [50]:
# def exit_strategy2(df):
#     filtered_df_list = []

#     # Group by each date
#     for date, group in df.groupby('date'):

#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue
            
#         # Step 1: Get the first 'selling_call_price' value for the date (first_call)
#         first_call = group['call_price'].iloc[0]

#         # Create columns for decay values 30, 40, 50
#         decay_levels = [0.3]
    
#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         for decay in decay_levels:
#             decay_loss_level = decay * first_call
#             decay_loss_triggered = False
#             group[f'call_decay_price_{int(decay * 100)}'] = np.nan
            
#             for i in range(len(subset)):
        
#                 if not decay_loss_triggered and subset['call_price'].iloc[i] < decay_loss_level:
#                     decay_loss_triggered = True
#                     a = i
#                     # # Additional condition for 30% decay: time after 14:00
#                     # if decay == 0.3 and subset['time'].iloc[i] > pd.Timestamp('14:00').time():
#                     #     decay_loss_triggered = False  # Reset trigger for 30% decay
                
#                 # Forward-fill the decay-loss level if triggered
#                 if decay_loss_triggered:
#                     group[f'call_decay_price_{int(decay * 100)}'] = subset.loc[subset.index[a], 'call_price']

#         # Append the adjusted group to the filtered list
#         filtered_df_list.append(group)

#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [51]:
# Step 5: Apply the exit strategy
df7 = exit_strategy2(df6)

In [52]:
df7.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time,put_stop_time,call_decay_price,call_decay_time
0,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,2686.00,2018-01-02 09:44:00,2684.7321,2681.75,3.95,2.675,09:44:00,NaN,NaN,NaN
1,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,2685.75,2018-01-02 09:45:00,2684.8000,2681.75,3.95,2.675,09:44:00,NaN,NaN,NaN
2,2018-01-02 09:46:00,2685.0,3.6,3.40,3.1,3.30,2686.09,2686.35,2685.17,2685.17,...,2684.50,2018-01-02 09:46:00,2684.7812,2681.75,3.95,2.675,09:44:00,NaN,NaN,NaN
3,2018-01-02 09:47:00,2685.0,4.1,3.00,3.6,2.80,2684.94,2685.85,2684.94,2685.75,...,2685.50,2018-01-02 09:47:00,2684.8235,2681.75,3.95,2.675,09:44:00,NaN,NaN,NaN
4,2018-01-02 09:48:00,2685.0,3.8,3.00,3.5,2.85,2685.74,2685.90,2685.27,2685.62,...,2685.50,2018-01-02 09:48:00,2684.8611,2681.75,3.95,2.675,09:44:00,NaN,NaN,NaN


In [53]:
# def exit_strategy3(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):

#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue

#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         # Step 3: Apply the stop-loss logic
#         #condition_triggered = False
#         group['call_condition_price'] = np.nan
#         group['call_firstlow_time'] = np.nan
#         group['call_firstcrossover_time'] = np.nan
#         group['call_secondlow_time'] = np.nan
#         group['call_secondcrossover_time'] = np.nan
    
#         min_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_firstlow_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_firstlow_time'].iloc[0]]
#         min_indices = subset[subset['futures_Close'] > subset['intraday_sma_20']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_firstcrossover_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_firstcrossover_time'].iloc[0]]
#         min_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_secondlow_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_secondlow_time'].iloc[0]]
#         min_indices = subset[subset['futures_Close'] > subset['intraday_sma_20']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_secondcrossover_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         group['call_condition_price'] = subset.loc[min_indices, 'call_price'].iloc[0]        
        
#         # Step 4: Add the adjusted group to the filtered list
#         filtered_df_list.append(group)
    
#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [54]:
def exit_strategy3(df):
    filtered_df_list = []

    # Group by each date
    for date, group in df.groupby('date'):

        # If all 'put_stop_time' values are NaN, skip the group
        if group['put_stop_time'].isna().all():
            filtered_df_list.append(group)  # Append the entire group unchanged
            continue

        # Initialize new columns with NaN
        group['call_condition_price'] = np.nan
        group['call_firstlow_time'] = np.nan
        group['call_firstcrossover_time'] = np.nan
        group['call_secondlow_time'] = np.nan
        group['call_secondcrossover_time'] = np.nan

        # Filter rows where time is after the first 'put_stop_time'
        put_stop_time = group['put_stop_time'].iloc[0]
        subset = group[group['time'] > put_stop_time]

        # Step 1: First low after put stop time
        first_low_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].index
        if not first_low_indices.empty:
            first_low_time = subset.loc[first_low_indices[0], 'time']
            group['call_firstlow_time'] = first_low_time
        else:
            filtered_df_list.append(group)
            continue

        # Step 2: First crossover after the first low
        subset_after_first_low = subset[subset['time'] > first_low_time]
        first_crossover_indices = subset_after_first_low[subset_after_first_low['futures_Close'] > subset_after_first_low['intraday_sma_20']].index
        if not first_crossover_indices.empty:
            first_crossover_time = subset_after_first_low.loc[first_crossover_indices[0], 'time']
            group['call_firstcrossover_time'] = first_crossover_time
        else:
            filtered_df_list.append(group)
            continue

        # Step 3: Second low after the first crossover
        subset_after_first_crossover = subset_after_first_low[subset_after_first_low['time'] > first_crossover_time]
        second_low_indices = subset_after_first_crossover[subset_after_first_crossover['futures_Low'] == subset_after_first_crossover['futures_Low_min']].index
        if not second_low_indices.empty:
            second_low_time = subset_after_first_crossover.loc[second_low_indices[0], 'time']
            group['call_secondlow_time'] = second_low_time
            #group['call_condition_price'] = subset_after_first_crossover.loc[second_low_indices[0], 'call_price']
        else:
            filtered_df_list.append(group)
            continue

        # Step 4: Second crossover after the second low
        subset_after_second_low = subset_after_first_crossover[subset_after_first_crossover['time'] > second_low_time]
        second_crossover_indices = subset_after_second_low[subset_after_second_low['futures_Close'] > subset_after_second_low['intraday_sma_20']].index
        if not second_crossover_indices.empty:
            second_crossover_time = subset_after_second_low.loc[second_crossover_indices[0], 'time']
            group['call_secondcrossover_time'] = second_crossover_time

            # Call price at second crossover
            group['call_condition_price'] = subset_after_second_low.loc[second_crossover_indices[0], 'call_price']

        # Append the processed group
        filtered_df_list.append(group)

    # Combine all filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [55]:
# Step 5: Apply the exit strategy
df8 = exit_strategy3(df7)

In [56]:
df8.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,selling_put_price,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time
0,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,2.675,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,2.675,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02 09:46:00,2685.0,3.6,3.40,3.1,3.30,2686.09,2686.35,2685.17,2685.17,...,2.675,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-02 09:47:00,2685.0,4.1,3.00,3.6,2.80,2684.94,2685.85,2684.94,2685.75,...,2.675,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-02 09:48:00,2685.0,3.8,3.00,3.5,2.85,2685.74,2685.90,2685.27,2685.62,...,2.675,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
def calculate_call_final_daily(df):
    # Initialize `call_final` column with NaN
    df['call_final'] = np.nan

    # Group by date and apply the logic
    for date, group in df.groupby('date'):
        call_decay_time = group['call_decay_time'].iloc[0]  # Assuming one value for the day
        call_secondcrossover_time = group['call_secondcrossover_time'].iloc[0]  # Assuming one value for the day

        if pd.isna(call_decay_time) and pd.isna(call_secondcrossover_time):
            # Both are NaN; use call_price for all rows
            df.loc[group.index,'call_final'] = group['call_price']
        elif pd.isna(call_decay_time):
            # Only call_decay_time is NaN; use call_condition_price
            df.loc[group.index, 'call_final'] = group['call_condition_price']
        elif pd.isna(call_secondcrossover_time):
            # Only call_secondcrossover_time is NaN; use call_decay_price
            df.loc[group.index, 'call_final'] = group['call_decay_price']
        else:
            # Both values are non-NaN
            if call_decay_time < call_secondcrossover_time:
                df.loc[group.index, 'call_final'] = group['call_decay_price']
            else:
                df.loc[group.index, 'call_final'] = group['call_condition_price']

    # Fill any remaining NaN values with call_price as a fallback
    #df['call_final'] = df['call_final'].fillna(df['call_price'])
    return df

In [58]:
# Apply the function to df8
df8 = calculate_call_final_daily(df8)

In [59]:
# Calculate the number of unique days
num_unique_days = df8['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1101


In [60]:
df8.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-01-02 09:44:00,2685.0,4.3,2.80,3.6,2.55,2686.96,2686.99,2686.24,2686.38,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.95
1,2018-01-02 09:45:00,2685.0,4.2,2.95,3.7,2.70,2686.39,2686.72,2685.92,2686.19,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.95
2,2018-01-02 09:46:00,2685.0,3.6,3.40,3.1,3.30,2686.09,2686.35,2685.17,2685.17,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.35
3,2018-01-02 09:47:00,2685.0,4.1,3.00,3.6,2.80,2684.94,2685.85,2684.94,2685.75,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.85
4,2018-01-02 09:48:00,2685.0,3.8,3.00,3.5,2.85,2685.74,2685.90,2685.27,2685.62,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.65


In [61]:
# def exit_strategy3(df):
#     filtered_df_list = []

#     # Group by each date
#     for date, group in df.groupby('date'):
        
#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue
            
#         # Step 1: Get the first 'selling_call_price' value for the date (first_call)
#         first_call = group['selling_call_price'].iloc[0]

#         # Create columns for stop values 20, 30, 40, 50, 60, 70
#         stop_levels = [1.2, 1.3, 1.4, 1.5, 1.6, 1.7]

#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         for stop in stop_levels:
#             stop_loss_level = stop * first_call
#             stop_loss_triggered = False
#             group[f'call_stop_price_{int(stop * 100)}'] = np.nan
            
#             for i in range(len(subset)):
#                 if not stop_loss_triggered and subset['call_price'].iloc[i] >= stop_loss_level:
#                     stop_loss_triggered = True
#                     a = i

#                 # Forward-fill the stop-loss level if triggered
#                 if stop_loss_triggered:
#                     group[f'call_stop_price_{int(stop * 100)}'] = subset.loc[subset.index[a], 'call_price']

#         # Append the adjusted group to the filtered list
#         filtered_df_list.append(group)

#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [62]:
# def exit_strategy3(df):
#     filtered_df_list = []

#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Skip days where 'put_stop_time' is NaN
#         if group['put_stop_time'].isna().all():
#             continue

#         # Step 1: Get the first 'call_price' value for each date
#         first_call = group['call_price'].iloc[0]

#         # Step 2: Calculate the condition and filter the rows accordingly
#         condition = group['call_price'] < 1.2 * first_call

#         # Step 3: Find the first row where the condition fails and filter accordingly
#         if condition.any():
#             first_fail_idx = condition[~condition].index[0] if (~condition).any() else None

#             # Keep rows before the first failure (inclusive of the row where it fails)
#             if first_fail_idx:
#                 filtered_group = group.loc[:first_fail_idx]
#             else:
#                 filtered_group = group  # Keep the entire group if the condition never fails

#             filtered_df_list.append(filtered_group)

#     return pd.concat(filtered_df_list, ignore_index=True)

In [63]:
# # Step 5: Apply the exit strategy
# df7 = exit_strategy3(df7)

In [64]:
df8 = df8.groupby('date').tail(1).reset_index(drop=True)

In [65]:
df8.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-01-02 10:19:00,2685.0,7.2,1.1,6.7,1.00,2690.22,2690.71,2690.22,2690.67,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95
1,2018-01-03 10:01:00,2700.0,5.9,1.1,5.1,1.00,2703.97,2704.63,2703.82,2704.59,...,09:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.50
2,2018-01-05 15:30:00,2730.0,9.6,0.1,7.8,0.05,2738.50,2738.62,2738.47,2738.62,...,11:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.70
3,2018-01-08 13:18:00,2740.0,8.0,0.2,6.3,0.10,2745.53,2746.47,2745.49,2746.47,...,11:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.15
4,2018-01-10 13:55:00,2745.0,6.8,0.4,5.4,0.30,2749.79,2750.80,2749.79,2750.80,...,10:43:00,10:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.10


In [66]:
df8.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'futures_Open', 'futures_High', 'futures_Low', 'futures_Close',
       'futures_Date_plus_1min', 'intraday_sma_20', 'futures_Low_min',
       'selling_call_price', 'selling_put_price', 'selling_time',
       'put_stop_time', 'call_decay_price', 'call_decay_time',
       'call_condition_price', 'call_firstlow_time',
       'call_firstcrossover_time', 'call_secondlow_time',
       'call_secondcrossover_time', 'call_final'],
      dtype='object')

In [67]:
# Calculate the number of unique days
num_unique_days = df8['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1101


In [68]:
# decay_levels = [0.3, 0.4, 0.5]

# for decay in decay_levels:
#     df7[f'call_pnl_{int(decay * 100)}'] = df7.apply(
#         lambda row: (
#             row['selling_call_price'] - row['call_price']
#         ) if pd.isna(row[f'call_decay_price_{int(decay * 100)}']) else (
#             row['selling_call_price'] - row[f'call_decay_price_{int(decay * 100)}']
#         ),
#         axis=1
#     )

In [69]:
df8.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-01-02 10:19:00,2685.0,7.2,1.1,6.7,1.00,2690.22,2690.71,2690.22,2690.67,...,09:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95
1,2018-01-03 10:01:00,2700.0,5.9,1.1,5.1,1.00,2703.97,2704.63,2703.82,2704.59,...,09:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.50
2,2018-01-05 15:30:00,2730.0,9.6,0.1,7.8,0.05,2738.50,2738.62,2738.47,2738.62,...,11:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.70
3,2018-01-08 13:18:00,2740.0,8.0,0.2,6.3,0.10,2745.53,2746.47,2745.49,2746.47,...,11:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.15
4,2018-01-10 13:55:00,2745.0,6.8,0.4,5.4,0.30,2749.79,2750.80,2749.79,2750.80,...,10:43:00,10:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.10


In [70]:
df8['call_pnl'] = df8['selling_call_price'] - df8['call_final']
df8['put_pnl'] = df8['selling_put_price'] - df8['put_price']

In [71]:
df8 = df8.sort_values(by='quote_datetime').reset_index(drop=True)

In [72]:
df8 = df8[df8['call_price'] >= 0]

In [73]:
df8 = df8[df8['put_price'] >= 0]

In [74]:
# df_filtered1['time'] = df_filtered1['quote_datetime'].dt.time

In [75]:
# df8['straddle_pnl_bps'] =  (df8['straddle_pnl']/ df8['spot932'])*10000
df8['put_pnl_bps'] =  (df8['put_pnl']/ df8['futures_Close'])*10000
df8['call_pnl_bps'] =  (df8['call_pnl']/ df8['futures_Close'])*10000

# for stop in stop_levels:
#     df8[f'call_stop_pnl_bps_{int(stop * 100)}'] = (df8[f'call_stop_pnl_{int(stop * 100)}']/ df8['futures_Close'])*10000

In [76]:
# df5 = df8[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'Close', 'spot932', 'gap931', 'realized_volatility_post30min', 'prev_close', 'PDL', 'PDH', 'gap', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'intraday_sma_11', 'below_sma_50', 'day_close', 'straddle932', 'call932', 'call_price', 'call_pnl', 'call_pnl_bps', 'put932', 'put_stop1_price', 'put_price', 'put_stoploss_time1', 'put_pnl', 'put_pnl_bps']]

In [77]:
df9 = df8[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'futures_Close', 'spot932', 'gap931', 'realized_volatility_post30min', 'prev_close', 'gap', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50', 'day_close', 'straddle932', 'straddle_exit', 'selling_time', 'put932', 'selling_put_price', 'put_price', 'put_stop_time', 'put_pnl', 'put_pnl_bps', 'call932', 'selling_call_price', 'call_price', 'call_decay_price', 'call_decay_time',
       'call_condition_price', 'call_firstlow_time', 'call_firstcrossover_time', 'call_secondlow_time', 'call_secondcrossover_time', 'call_final', 'call_pnl', 'call_pnl_bps']]

In [78]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1101


In [79]:
df9['year'] = df9['quote_datetime'].dt.year

In [80]:
# df6 = df5.groupby('date').tail(1).reset_index(drop=True)

In [81]:
df9 = df9[df9['below_sma_50'] == 'Yes']

In [82]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

329


In [83]:
# df5['total_pnl_bps'] = df5['call_pnl_bps'] + df5['put_pnl_bps']

In [84]:
# df9 = df9[(df9['below_sma_50'] == 'Yes') & (df9['prev_day_neg'] == 'yes')]

In [85]:
# df9 = df9[df9['gap']/df9['straddle932'] < 1]

In [86]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

329


In [87]:
df10 = df9.copy()

In [88]:
# # Convert '10:32' to a datetime.time object
# time_limit = datetime.strptime('10:32', '%H:%M').time()

# # Filter the DataFrame
# df10 = df9[df9['selling_time'] <= time_limit]

In [89]:
# Calculate the number of unique days
num_unique_days = df10['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

329


In [90]:
df10['final_pnl_bps'] = df10['call_pnl_bps'] + df10['put_pnl_bps']

In [91]:
# df10['final_pnl_bps120'] = df10['call_stop_pnl_bps_120'] + df10['put_pnl_bps']
# df10['final_pnl_bps130'] = df10['call_stop_pnl_bps_130'] + df10['put_pnl_bps']
# df10['final_pnl_bps140'] = df10['call_stop_pnl_bps_140'] + df10['put_pnl_bps']
# df10['final_pnl_bps150'] = df10['call_stop_pnl_bps_150'] + df10['put_pnl_bps']
# df10['final_pnl_bps160'] = df10['call_stop_pnl_bps_160'] + df10['put_pnl_bps']
# df10['final_pnl_bps170'] = df10['call_stop_pnl_bps_170'] + df10['put_pnl_bps']

In [92]:
# Initialize an empty DataFrame to store yearly drawdown information
yearly_drawdowns = []

# Group the DataFrame by year and calculate the drawdowns
for year, group in df10.groupby('year'):
    # Calculate the cumulative sum of pnl_bps
    group['cumulative_pnl'] = (group['final_pnl_bps']).cumsum()
    
    # Calculate the running maximum of the cumulative pnl
    group['running_max'] = group['cumulative_pnl'].cummax()
    
    # Calculate the drawdown as the difference between the running max and the current cumulative pnl
    group['drawdown'] = group['running_max'] - group['cumulative_pnl']
    
    # Find the maximum drawdown for the year
    max_drawdown = group['drawdown'].max()
    
    # Append the results to the list
    yearly_drawdowns.append({'year': year, 'max_drawdown': max_drawdown})

# Create a DataFrame from the yearly drawdown information
yearly_drawdowns_df = pd.DataFrame(yearly_drawdowns)

print(yearly_drawdowns_df)

   year  max_drawdown
0  2018       43.6927
1  2019       91.3539
2  2020      262.6133
3  2021       82.5257
4  2022      295.6573
5  2023       93.3294
6  2024      161.2993


In [93]:
# Calculate the number of unique days
num_unique_days = df10['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

329


In [94]:
df10.to_excel('call50stopdecay50smaput50_fut09_10322_marker2.xlsx')